In [12]:
from dash import dcc, html, Dash, dash_table, Input, Output,State
import yfinance as yf
import plotly.graph_objs as go

In [13]:
ticker = "APP"
price = yf.Ticker(ticker).history(period='1d', interval='15m').reset_index()

app = Dash()
app.layout = html.Div([
    html.H1('Stock Dashboard'),
    dcc.Input(
        id='ticker-input',
        placeholder='Enter ticker symbol',
        style={'width': '50%' }
    ),
    html.Button('Submit', id='submit-button'),
    html.Br(),
    html.Br(),
    dcc.Tabs([
        dcc.Tab(label='Candle Stick Chart',
                children = dcc.Graph(id='stock-graph')

            ),
        dcc.Tab(label='Data',
                children=
                    [html.Div(id = 'latest-price-div'),
                     dash_table.DataTable(id = 'stock-data'

                                         )]


    ),
    dcc.Interval(id = 'chart-interval', interval = 1000 * 60 * 15, n_intervals = 0),
    dcc.Interval(id = 'data-interval', interval = 1000 * 60, n_intervals = 0)


])])

@app.callback(
    Output('stock-graph', 'figure'),
    Input('submit-button', 'n_clicks'),
    Input('chart-interval', 'n_intervals'),
    State('ticker-input', 'value')
)

def update_chart(button_click, chart_interval, ticker):
    if ticker is None:
        return {}
    else :
        price = yf.Ticker(ticker).history(period='1d', interval= '15m').reset_index()
        if len(price) > 0 :
            fig = go.Figure(data = go.Candlestick(
                             x = price['Datetime'],
                            open = price['Open'],
                            high = price['High'],
                            low = price['Low'],
                            close = price['Close']
                            )
            )
            return fig
        else :
            return {}


@app.callback(
    Output('latest-price-div','children'),
    Output('stock-data', 'data'),
    Input('submit-button', 'n_clicks'),
    Input('data-interval', 'n_intervals'),
    State('ticker-input', 'value')
)

def update_table(button_click, data_interval,  ticker):
    if ticker is None:
        return '',{}
    else :
        price = yf.Ticker(ticker).history(period='1d', interval= '1m').reset_index().tail(10)
        if len(price) > 0 :
            latest_price = price['Close'].iloc[-1]
            latest_time = price['Datetime'].max().strftime('%b %d, %Y %I:%M:%S %p')
            return f'The latest price at {latest_time} is: {latest_price}', price.to_dict('records')
        else :
            return f'No data for ticker {ticker} on Yahoo Finance', {}


In [14]:
if __name__ == '__main__':
    app.run(debug=True)